In [7]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import font_manager
from matplotlib import rcParams
font_manager.findSystemFonts(fontpaths=None, fontext="ttf")

font_dir = ['C:\\Windows\\Fonts']
for font in font_manager.findSystemFonts(font_dir):
    font_manager.fontManager.addfont(font)
rcParams['font.family'] = 'Ancizar Sans'


In [8]:

def read_data(filename):
    data = np.loadtxt('../data/'+filename+'.txt', delimiter=' ')
    return data


In [9]:

#Data files belong to cold dark matter halo simulations of Milky Way particles interacting with the Large Magellanic Cloud (LMC)
#mw prefix is for MW particles in the non interacting regime
#lmc prefix is for MW particles in perturbed dark matter halo

data_mw = read_data('rand_mwb1_000')
data_lmc = read_data('rand_mwlmcb1_110')

#Potential energy of particles in dark matter halo
pot_mw = data_mw[:,6]
pot_mw_lmc = data_lmc[:,6]

#Extract velocity magnitude of particles in dark matter halo
v_mw = data_mw[:,3:6]
v_mw_lmc = data_lmc[:,3:6]

#Extract index of particles
idx_mw = data_mw[:,7]
idx_lmc = data_lmc[:,7]
def calculate_v_mag_square(v_mw):
    v_mag = np.sqrt(v_mw[:,0]**2+v_mw[:,1]**2+v_mw[:,2]**2)
    return v_mag

v_mag_square_mw = calculate_v_mag_square(v_mw)
v_mag_square_mw_lmc = calculate_v_mag_square(v_mw_lmc)

E_mw = pot_mw+v_mag_square_mw
E_mw_lmc = pot_mw_lmc+v_mag_square_mw_lmc

#Positions and velocities of Milky Way galaxy dark matter halo particles

pos_mw = data_mw[:,0:3]
pos_mw_lmc = data_lmc[:,0:3]

vel_mw = data_mw[:,3:6]
vel_mw_lmc = data_lmc[:,3:6]


In [10]:

#Calculate angular momentum components

def angular_momentum(pos, vel):
    """Calculate angular momentum components of particles given position an velocity arrays.

    Args:
        pos (_array_): _Position 3 dim_
        vel (_array_): _Velocities 3 dim_

    Returns:
        _1d array_: Angular momenta in each direction and total mag__
    """
    L = np.cross(pos, vel)
    mag = np.linalg.norm(L, axis=1)
    return L[:,0], L[:,1], L[:,2], mag

Lx1, Ly1, Lz1, L_mag_mw = angular_momentum(pos_mw, vel_mw)


In [11]:

print(len(Lx1))
print(pos_mw[:,0])
Lx2, Ly2, Lz2, L_mag_mw_lmc = angular_momentum(pos_mw_lmc, vel_mw_lmc)

pos_mag_mw = np.linalg.norm(pos_mw, axis=1)
pos_mag_lmc = np.linalg.norm(pos_mw_lmc, axis=1)


1000000
[ 2.02146411e-01 -1.94147987e+01  1.38229462e+02 ... -2.45280045e+02
  1.82924179e+02  3.10036072e+02]


In [18]:


#Range and step def for distance and angular momentum arrays

r =  np.arange(50,250,10)
L = np.arange(9,80000,2500)
hist_r_L = np.zeros((len(r),len(L)))
for i in range(0,len(r)-1):
    r_index = np.where((pos_mag_mw > r[i]) & (pos_mag_mw[:,2] < r[i+1]))
    for j in range(0,len(L)-1):
        Nl = np.where((Lz1[r_index]> L[j]) & (Lz1[r_index] < L[j+1]))
        hist_r_L[i,j] = np.shape(Nl)[1]

fig,ax = plt.subplots(1,1,figsize=(10,10))
ax.set_ylabel('L especifico en y')
ax.set_xlabel('Rango de distancias en kpc')
im = plt.imshow(np.log10(hist_r_L.T), extent=[50, 250, 0, 90000], aspect='auto', origin='lower', vmin=0, vmax=5,cmap='Greys')
plt.colorbar()


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:


# low_L = np.where(L_mag_mw < 5000)
low_L = np.where (Lz1 < 10000)
high_L = np.where(Lz1 > 10000)

# im = plt.imshow(pos_mag_mw[low_L], L_mag_mw[low_L],  origin='lower', vmin=0, vmax=5,cmap='Greys')

# print(low_L[0], medium_L[0], high_L[0])
def hist_L_r(pos,ang_m):
    r =  np.arange(50,250,10)
    L = np.arange(9,50000,2500)
    hist_r_L = np.zeros((len(r),len(L)))
    for i in range(0,len(r)-1):
        r_index = np.where((pos > r[i]) & (pos < r[i+1]))
        for j in range(0,len(L)-1):
            Nl = np.where((ang_m[r_index]> L[j]) & (ang_m[r_index] < L[j+1]))
            hist_r_L[i,j] = np.shape(Nl)[1]
    fig, ax = plt.subplots(1,1,figsize=(10,10))
    im = plt.imshow(np.log10(hist_r_L.T), extent=[50, 250, 0, 50000], aspect='auto', origin='lower', vmin=0, vmax=5,cmap='Greys')
    ax.set_xlabel(r'x [$\mathrm{kpc}$]', fontsize=20)
    ax.set_ylabel(r'y [$\mathrm{kpc}$]', fontsize=20)
    # major_ticks = np.arange(0, 250, 10)
    # minor_ticks = np.arange(0, 250, 2)
    # ax.set_xticks(major_ticks)
    # ax.set_xticks(minor_ticks, minor=True)
    # ax.set_yticks(major_ticks)
    # ax.set_yticks(minor_ticks, minor=True)
    # # ax.grid(which='both')
    # ax.grid(which='minor', alpha=0.2)
    # ax.grid(which='major', alpha=0.5)
    ax.set_title("Proyección cartesiana en el plano xyu", fontsize=35, y=1.05)
    plt.colorbar()
    return hist_r_L

# def regions(pos):
#     L1 = pos +5000
#     L2 = pos +10000
#     return L1, L2
# lines_mw = regions(pos)
# L1 = lines_mw[0]
# L2 = lines_mw[1]
# ind = []
# for i in range(0,len(pos)):
#     if ang_m > L1[i] and ang_m[i] < L2[i]:
#         ind.append(i)
# print(len((low_L)))

print(len(pos_mw_lmc[:,0][low_L]))
pos_mw_lmc_z = pos_mw_lmc[:,2]
# hist_L_r(pos_mag_mw[low_L], L_mag_mw[low_L])
# hist_L_r(pos_mag_mw[medium_L], L_mag_mw[medium_L])
# hist_L_r(pos_mag_mw[high_L], L_mag_mw[high_L])

hist_L_r(pos_mw_lmc_z[low_L], Lz2[low_L])
# hist_L_r(pos_mag_lmc[medium_L], L_mag_mw_lmc[medium_L])
# hist_L_r(pos_mag_lmc[high_L], L_mag_mw_lmc[high_L])
